In [15]:
from dataloader import GcMTLDataset
import pandas as pd
import torch

label_path = '/home/huangjialong/projects/BiomedCLIP-PUNCE/datatools/gc/n-labels/all_label.csv'
df = pd.read_csv(label_path, header=None)
keep_same_psize = 1000
file_name = df.iloc[:, 0].values
file_label = df.iloc[:, 1].values
root = '/data/wsi/TCTGC2625-features/plip-fine'
num_classes = [5]
num_task = 0
fine_concat = True
dataset = GcMTLDataset(file_name, file_label,root, num_task, num_classes, fine_concat=fine_concat, keep_same_psize=keep_same_psize)
# dataset = GcMTLFineDataset(file_name, file_label,root, num_task, num_classes, fine_concat=fine_concat)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=100, shuffle=False, num_workers=0, pin_memory=True)
for i, (f, l, t, f) in enumerate(data_loader):
    print(l.shape)
    print(t)

torch.Size([9])
tensor([1, 1, 0, 1, 2, 2, 2, 2, 2])


In [18]:
import torch
import pandas as pd
import shutil, os
from tqdm import tqdm

fn = '/data/wsi/TCTGC50k-labels/TCTGC50k-volume1-labels-path.csv'
df = pd.read_csv(fn)
label_wsi_names = df['wsi_name'].tolist()
label_wsi_names = [i.replace('.pt', '') for i in label_wsi_names]
print(len(label_wsi_names))
wsi_path = '/data/wsi/TCTGC50k/TCTGC50k-volume1'
output_wsi_path = '/data/wsi/TCTGC50k/TCTGC50k-volume-deprecated13'
wsi_names = os.listdir(wsi_path)
diff = set(wsi_names) - set(label_wsi_names)
diff = list(diff)
len(diff)

9854


0

In [17]:
# for i in diff:
#     shutil.move(os.path.join(wsi_path, i), os.path.join(output_wsi_path, i))

In [2]:
import torch
from model import MIL_MTL

def count_parameters(model):
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    params /= 1024 ** 2
    return params

num_task = 3
num_classes=[1,5,3]
input_dim = 256
model2 = MIL_MTL(num_classes=num_classes, num_task=num_task, input_dim=input_dim)
print((count_parameters(model2)))

size = (5, 10, 256) # [B, N, C]
mask = torch.ones(size[0], size[1], size[2])
a = torch.randn(size)
task_id = torch.randint(0, num_task, (size[0],))
output = model2(a, task_id)
print(output.shape)
# print(output.shape)

0.19251346588134766
torch.Size([5, 9])


In [1]:
from utils import multi_class_scores_mtl
import numpy as np

n = 100
n_classes = 9
bag_labels = np.random.randint(0, n_classes, n)
class_labels = [['NILM', 'ASC-US', 'LSIL', 'ASC-H', 'HSIL', 'AGC', 'BV', 'M', 'T']]
bag_logits = np.random.rand(*(n, n_classes))
wsi_names = ['WSI' + str(i) for i in range(n)]
threshold = 0.5

multi_class_scores_mtl(bag_labels, bag_logits, class_labels, wsi_names, threshold, eval_only=False)
# def multi_class_scores_mtl(bag_labels, bag_logits, class_labels, wsi_names, threshold, eval_only):

(68, 9)
[ERROR] WSI41 risk prediction is wrong: [0.9613, 0.5322, 0.1481, 0.7097, 0.3702, 0.3584, 0.9704, 0.854, 0.2885]
[ERROR] WSI55 risk prediction is wrong: [0.9566, 0.5628, 0.1546, 0.5229, 0.6502, 0.7161, 0.5364, 0.5956, 0.8355]
[INFO] confusion matrix for cancer labels:
Confusion Matrix for 68 data
+-----------+------+--------+------+-------+------+-----+------+
| 实际\预测 | NILM | ASC-US | LSIL | ASC-H | HSIL | AGC | 总计 |
+-----------+------+--------+------+-------+------+-----+------+
|    NILM   |  0   |   12   |  0   |   0   |  0   |  0  |  12  |
|   ASC-US  |  0   |   8    |  0   |   0   |  0   |  0  |  8   |
|    LSIL   |  1   |   9    |  0   |   0   |  0   |  0  |  10  |
|   ASC-H   |  1   |   16   |  0   |   0   |  0   |  0  |  17  |
|    HSIL   |  0   |   13   |  0   |   0   |  0   |  0  |  13  |
|    AGC    |  0   |   8    |  0   |   0   |  0   |  0  |  8   |
|    总计   |  2   |   66   |  0   |   0   |  0   |  0  |  68  |
+-----------+------+--------+------+-------+------+--

/home/huangjialong/anaconda3/envs/biomed/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.45560977424582605,
 0.06,
 0.06298076923076923,
 0.07291666666666666,
 0.06665393430099313)

In [3]:
import numpy as np

# 示例数组
array = np.array([1, 2, 3])

# 想要添加的新元素
new_elements = np.array([4, 5, 6, 7])

print(array.shape)
# 使用numpy.concatenate将新元素添加到数组末尾
extended_array = np.concatenate((array, new_elements))
extended_array


(3,)


array([1, 2, 3, 4, 5, 6, 7])